# ES6 规范中的词法环境与作用域、闭包的关系


> [ES6 规范对词法环境（Lexical Environment）的描述](https://262.ecma-international.org/6.0/#sec-lexical-environments)

词法环境（Lexical Environment）可看作是一种用于记录变量和函数声明的数据结构。这个结构包括两个主要部分：

- 环境记录：用于存储变量和函数声明。
- 对外部词法环境的引用：指向父级词法环境。

**词法环境有三种**

- 全局环境：当一个 JS 程序被加载的时候，会创建一个全局环境，它存在于 JS 程序整个生命周期，
  全局变量与全局函数声明都存储在全局环境的环境记录中。
  全局环境的环境记录还与全局对象（浏览器 `window` 或 Node `global` 等）关联，在整个 JS 程序中可被访问。
  全局环境的对外引用为 `null`，因为全局环境是 JS 程序的唯一最外层词法环境。

- 函数环境：当一个函数被调用的时候，会创建一个函数环境，它与函数生命周期通常是同步的，但也有例外，就是「闭包」。
  函数的参数、局部变量和内部的函数声明存储于函数环境的环境记录中。
  函数环境的父级词法环境为函数声明时所在的词法环境，也就是它定义的地方。

- 模块环境：ES 模块分「静态导入」（`import`）和「动态导入」（`import()`），两者的词法环境略有不同。

  静态导入的模块首次被加载的时候，会创建一个模块环境，模块的导出内容会被缓存。此后在 JS 程序整个生命周期中再次导入该模块，会使用模块缓存的内容，不会重新执行模块代码，也不会重新创建模块环境。

  动态导入的模块每次都会创建一个模块环境，但当动态导入的模块不再被引用的时候，对应的模块环境也会被回收。

  模块内的局部变量、函数声明、导出和导入都存储于模块环境的环境记录中。
  模块环境的对外引用为全局环境。

> 「加载」通常指的是 JS 代码被读取到内存中的过程。

**闭包** 是指函数及其引用的词法环境的组合。
如果一个函数产生了闭包，当它执行完，它的词法环境依然被闭包所引用，不会被内存回收。看例子：


In [5]:
function createCounter() {
  let count = 0;

  // 匿名函数引用了外层函数 createCounter 的变量 count，匿名函数和 createCounter 的词法环境形成了闭包。
  return function () {
    count++;
    console.log(count);
  };
}
// 尽管函数 createCounter 执行完，但其词法环境依然被 counter 保留，因此 count 依然可以被访问。
const counter = createCounter();
counter();


1
2


环境记录有三种类型：

- 声明性环境记录：存储由 variable, constant, let, class, module, import, function declarations 等声明的标识符。
  - 函数环境记录：存储函数声明和函数的参数信息，如果该函数不是 ArrowFunction，则提供 this 绑定。
  - 模块环境记录：存储模块的导入和导出声明。
- 对象环境记录：存储由全局对象（如 window 或 global）。
- 全局环境记录：声明性环境记录 + 对象环境记录。

## 作用域

作用域是

作用域是词法环境的抽象，词法环境是作用域的实现，

作用域和词法环境不是一一对应关系的，有块级作用域但是没有块级词法环境。
块级作用域也是通过函数词法环境或全局词法环境实现的。
